In [125]:
class Sudoku:
    def __init__(self, string):
        board = [[],[],[],[],[],[],[],[],[]]  
        # For each of the numbers 1-9, filled holds the indices of the rows, columns and nonets that already contain that number, and the total number of cells holding that number
        # Row indices are labeled y; column indices labeled x; and nonets are indexed by row//3+(y//3)*3 
        filled = {value:{"rows": set(), "cols": set(), "nonets": set(), "count": 0} for value in range(1,10)}
        # For each type of grouping (row, column, or nonet), and each grouping of that type indexed 0-8, blanks holds the indices of blanks cells within that grouping 
        # Within rows, cells are indexed by column; within columns, indexed by row; and within nonets, indexed by x%3+(y%3)*3 
        blanks = {key: {index: set() for index in range(9)} for key in {"rows", "cols", "nonets"}}
        for y in range(9):
            x = 0
            # We slice the string into substrings of 9 characters which each correspond to a row
            for char in string[y*9:(y+1)*9]:
                value = int(char)
                board[y].append(value)
                if value == 0:
                    blanks["rows"][y].add(x) 
                    blanks["cols"][x].add(y) 
                    blanks["nonets"][x//3+(y//3)*3].add(x%3+(y%3)*3) 
                else:
                    filled[value]["rows"].add(y)
                    filled[value]["cols"].add(x)
                    filled[value]["nonets"].add(x//3+(y//3)*3)
                    filled[value]["count"] += 1
                x += 1
        self.board = board
        self.filled = filled
        self.blanks = blanks
        queue = [item[0] for item in sorted(filled.items(), key=lambda item: item[1]["count"], reverse=True)]
        self.queue = queue
        # print(queue)
        # for key in blanks.keys():
        #     print("{}: {}".format(key, blanks[key]))
        # for key in filled.keys():
        #     print("{}: {}".format(key, filled[key]))

def display(puzzle, new):
    for i in range(9):
        if i%3 == 0:
            print()
            if i != 0:
                print("_ "*11)
        for j in range(9):
            if j%9 == 0:
                print()
            if j%3 == 0 and j != 0:
                print("|", end = " ")
            if (j,i) in new:
                print('\033[1m' + str(puzzle.board[i][j]) + "\033[0;0m", end = " ")
            else:
                print(puzzle.board[i][j], end = " ")
    print()
    print("\n"+"* "*11+"\n")

In [126]:
easy = "800020910234510007710080054600100305185000720040602800068000400000000162000407530"
medium = "300007086005003000000005320940102050200090700850004000089000007070040000034801900"
hard = "000500900050020001300609000070180002105000087008004000004750000030208050000406020"
expert = "000500000005002940006000070000050020007004100800390000403000006000400007080060002"
evil = "080010000000000020006800045004100086000000700300009000400030057007000100000500200"
hardest_ever = "800000000003600000070090200050007000000045700000100030001000068008500010090000400"

easy_puzzle = Sudoku(easy)
display(easy_puzzle, new=[])



8 0 0 | 0 2 0 | 9 1 0 
2 3 4 | 5 1 0 | 0 0 7 
7 1 0 | 0 8 0 | 0 5 4 
_ _ _ _ _ _ _ _ _ _ _ 

6 0 0 | 1 0 0 | 3 0 5 
1 8 5 | 0 0 0 | 7 2 0 
0 4 0 | 6 0 2 | 8 0 0 
_ _ _ _ _ _ _ _ _ _ _ 

0 6 8 | 0 0 0 | 4 0 0 
0 0 0 | 0 0 0 | 1 6 2 
0 0 0 | 4 0 7 | 5 3 0 

* * * * * * * * * * * 



In [127]:
import queue


def easy_solve(puzzle):
    count = 0
    # Get the next value from the queue while it is non-empty 
    while len(puzzle.queue) > 0:
        value = puzzle.queue.pop(0)
        # print("**********************************************************")
        # print(value)
        # Discard the value and continue to next in queue if it has already been fully filled in
        if puzzle.filled[value]["count"] == 9:
            del puzzle.filled[value]
            # print("deleting ", value)
            continue
        # The set of all rows (& columns and nonets?) that contain blanks and do not contain the current value
        unfilled_rows = set(puzzle.blanks["rows"].keys()).difference(puzzle.filled[value]["rows"])
        unfilled_cols = set(puzzle.blanks["cols"].keys()).difference(puzzle.filled[value]["cols"])
        unfilled_nonets = set(puzzle.blanks["nonets"].keys()).difference(puzzle.filled[value]["nonets"])
        candidates = set()

        for y in unfilled_rows:
            # The set of column indices of blanks in the current row, excluding columns that contain the current value
            row_blanks = puzzle.blanks["rows"][y].difference(puzzle.filled[value]["cols"])
            # Filter out column indices of blanks whose nonet already contains the current value
            fillable_cols = set(filter(lambda x: x//3+(y//3)*3 not in puzzle.filled[value]["nonets"], row_blanks))
            # If we've narrowed down to a single candidate, add it to candidates
            if len(fillable_cols) == 1:
                x = fillable_cols.pop()
                candidates.add((x,y))

        for x in unfilled_cols:
            # The set of column indices of blanks in the current row, excluding columns that contain the current value
            col_blanks = puzzle.blanks["cols"][x].difference(puzzle.filled[value]["rows"])
            # Filter out column indices of blanks whose nonet already contains the current value
            fillable_rows = set(filter(lambda y: x//3+(y//3)*3 not in puzzle.filled[value]["nonets"], col_blanks))
            # If we've narrowed down to a single candidate, add it to candidates
            if len(fillable_rows) == 1:
                y = fillable_rows.pop()
                candidates.add((x,y))

        for n in unfilled_nonets:
            # The set of nonet indices of blanks in the current nonet
            nonet_blanks = puzzle.blanks["nonets"][n]
            # Filter out nonet indices of blanks whose row or column already contains the current value
            fillable_cells = set(filter(lambda m: (m//3+(n//3)*3 not in puzzle.filled[value]["rows"]) and (m%3+(n%3)*3 not in puzzle.filled[value]["cols"]), nonet_blanks))
            # If we've narrowed down to a single candidate, add it to candidates
            if len(fillable_cells) == 1:
                m = fillable_cells.pop()
                candidates.add((m%3+(n%3)*3,m//3+(n//3)*3))
                
        # print(candidates)
        if len(candidates) == 0:
            count += 1
        else:
            count = 0
            
        for (x,y) in candidates:
            puzzle.board[y][x] = value
            puzzle.blanks["rows"][y].remove(x)
            puzzle.blanks["cols"][x].remove(y)  
            puzzle.blanks["nonets"][x//3+(y//3)*3].remove(x%3+(y%3)*3)

            puzzle.filled[value]["count"] += 1
            puzzle.filled[value]["rows"].add(y)
            puzzle.filled[value]["cols"].add(x)       
            puzzle.filled[value]["nonets"].add(x//3+(y//3)*3)

        if puzzle.filled[value]["count"] == 9:
            del puzzle.filled[value]
            # print("deleting ", value)
        else:
            puzzle.queue.append(value)
        # display(puzzle, [])
        if count == 10:
            return False
    return True


In [ ]:
def recursive_solve(puzzle):
    

In [128]:
import time
start = time.time()
# puzzle = Sudoku(easy)
# puzzle = Sudoku(medium)
# puzzle = Sudoku(hard)
# puzzle = Sudoku(expert)
puzzle = Sudoku(evil)
# puzzle = Sudoku(hardest_ever)
display(puzzle, [])
easy_solve(puzzle)
display(puzzle, [])
end = time.time()
print(end-start," s")



0 8 0 | 0 1 0 | 0 0 0 
0 0 0 | 0 0 0 | 0 2 0 
0 0 6 | 8 0 0 | 0 4 5 
_ _ _ _ _ _ _ _ _ _ _ 

0 0 4 | 1 0 0 | 0 8 6 
0 0 0 | 0 0 0 | 7 0 0 
3 0 0 | 0 0 9 | 0 0 0 
_ _ _ _ _ _ _ _ _ _ _ 

4 0 0 | 0 3 0 | 0 5 7 
0 0 7 | 0 0 0 | 1 0 0 
0 0 0 | 5 0 0 | 2 0 0 

* * * * * * * * * * * 



0 8 0 | 0 1 0 | 6 7 9 
0 4 0 | 0 0 0 | 8 2 1 
0 0 6 | 8 0 0 | 3 4 5 
_ _ _ _ _ _ _ _ _ _ _ 

0 0 4 | 1 0 3 | 5 8 6 
0 0 0 | 0 0 0 | 7 9 3 
3 0 0 | 0 0 9 | 4 1 2 
_ _ _ _ _ _ _ _ _ _ _ 

4 0 0 | 0 3 0 | 9 5 7 
0 0 7 | 0 0 0 | 1 0 0 
0 0 0 | 5 0 0 | 2 0 0 

* * * * * * * * * * * 

0.002000093460083008  s
